In [1]:
#read file in dataframe
df_csv = spark.read.csv("/Users/mayam/Desktop/h516lab/2012-10-01.csv", header = 'true')
df_csv.show()

+----------+--------+-------+---------+------+---------+--------+-------+-------+-----+
|      date|    time|   size|r_version|r_arch|     r_os| package|version|country|ip_id|
+----------+--------+-------+---------+------+---------+--------+-------+-------+-----+
|2012-10-01|00:30:13|  35165|   2.15.1|  i686|linux-gnu|quadprog|  1.5-4|     AU|    1|
|2012-10-01|00:30:15| 212967|   2.15.1|  i686|linux-gnu|  lavaan|  0.5-9|     AU|    1|
|2012-10-01|02:30:16| 167199|   2.15.1|x86_64|linux-gnu| formatR|    0.6|     US|    2|
|2012-10-01|02:30:16|  21164|   2.15.1|x86_64|linux-gnu| stringr|  0.6.1|     US|    2|
|2012-10-01|02:30:13|  11046|   2.15.1|x86_64|linux-gnu|evaluate|  0.4.2|     US|    2|
|2012-10-01|02:30:13|  42294|   2.15.1|x86_64|linux-gnu|  digest|  0.5.2|     US|    2|
|2012-10-01|02:30:16| 435407|   2.15.1|x86_64|linux-gnu|   knitr|    0.8|     US|    2|
|2012-10-01|02:26:05| 326143|   2.15.1|  i686|linux-gnu|    mice|   2.13|     AU|    1|
|2012-10-01|02:21:09| 119459|   

In [2]:
#total count entries in data frame
df_csv.count()

159

In [74]:
#with reduceBy
rdd = sc.textFile ("/Users/mayam/Desktop/h516lab/2012-10-01.csv")
rdd_f = rdd.first()
header = sc.parallelize([rdd_f])
rdd1 = rdd.subtract (header)
rdd1.take(10)

['"2012-10-01","02:30:13",368192,"2.15.1","x86_64","linux-gnu","plyr","1.7.1","US",2',
 '"2012-10-01","12:43:46",2094435,"2.15.1","x86_64","linux-gnu","mosaic","0.6-2","US",2',
 '"2012-10-01","12:53:25",868687,"2.15.1","x86_64","linux-gnu","RCurl","1.95-0","US",2',
 '"2012-10-01","12:45:33",2094444,"2.15.1","x86_64","linux-gnu","mosaic","0.6-2","US",2',
 '"2012-10-01","12:43:54",868687,"2.15.1","x86_64","linux-gnu","RCurl","1.95-0","US",2',
 '"2012-10-01","13:17:45",868701,"2.15.1","x86_64","linux-gnu","RCurl","1.95-0","US",2',
 '"2012-10-01","14:32:43",868696,"2.15.1","x86_64","linux-gnu","RCurl","1.95-0","US",2',
 '"2012-10-01","14:44:34",8968,"2.15.1","x86_64","linux-gnu","bitops","1.0-4.1","US",2',
 '"2012-10-01","15:13:00",2094449,"2.15.1","x86_64","linux-gnu","mosaic","0.6-2","US",2',
 '"2012-10-01","16:13:15",868696,"2.15.1","x86_64","linux-gnu","RCurl","1.95-0","US",2']

In [73]:
grouped = rdd1.groupByKey()
grouped.map(lambda x:(x[6],))

PythonRDD[150] at RDD at PythonRDD.scala:53

In [3]:
#Reading data as SQL table
df_csv.createOrReplaceTempView("SQL_table")
df_csv.show()

+----------+--------+-------+---------+------+---------+--------+-------+-------+-----+
|      date|    time|   size|r_version|r_arch|     r_os| package|version|country|ip_id|
+----------+--------+-------+---------+------+---------+--------+-------+-------+-----+
|2012-10-01|00:30:13|  35165|   2.15.1|  i686|linux-gnu|quadprog|  1.5-4|     AU|    1|
|2012-10-01|00:30:15| 212967|   2.15.1|  i686|linux-gnu|  lavaan|  0.5-9|     AU|    1|
|2012-10-01|02:30:16| 167199|   2.15.1|x86_64|linux-gnu| formatR|    0.6|     US|    2|
|2012-10-01|02:30:16|  21164|   2.15.1|x86_64|linux-gnu| stringr|  0.6.1|     US|    2|
|2012-10-01|02:30:13|  11046|   2.15.1|x86_64|linux-gnu|evaluate|  0.4.2|     US|    2|
|2012-10-01|02:30:13|  42294|   2.15.1|x86_64|linux-gnu|  digest|  0.5.2|     US|    2|
|2012-10-01|02:30:16| 435407|   2.15.1|x86_64|linux-gnu|   knitr|    0.8|     US|    2|
|2012-10-01|02:26:05| 326143|   2.15.1|  i686|linux-gnu|    mice|   2.13|     AU|    1|
|2012-10-01|02:21:09| 119459|   

In [75]:
#1.Count the number of entries
df_sql1 = spark.sql("SELECT count(*) FROM SQL_table")
df_sql1.show()

+--------+
|count(1)|
+--------+
|     159|
+--------+



In [112]:
#4-2.Most popular package  without using “reduceByKey”
df_sql2 = spark.sql("SELECT count(*),package FROM SQL_table GROUP BY package order by count(*) desc")
df_sql2.first()

Row(count(1)=62, package='mosaic')

In [91]:
#4-3.Which country is using R the most without using “reduceByKey”
df_sql3 = spark.sql("SELECT count(*),country FROM SQL_table where r_version <> 'NA' GROUP BY country order by count(*) desc")
df_sql3.first()

Row(count(1)=152, country='US')

In [129]:
#5.What is the most popular package from Germany (DE)
df_sql4 = spark.sql("SELECT count(*),package FROM SQL_table where country = 'DE' GROUP BY package order by count(*) desc")
df_sql4.show()

+--------+-------+
|count(1)|package|
+--------+-------+
+--------+-------+



In [82]:
#6.What is the total size downloaded  in Canada (CA)
df_sql5 = spark.sql("SELECT country, sum(size) as  total_size_downloaded FROM SQL_table where country = 'CA' GROUP BY country order by count(*) desc")
df_sql5.show()

+-------+---------------------+
|country|total_size_downloaded|
+-------+---------------------+
+-------+---------------------+



In [80]:
#7.Which version was downloaded most between 12 noon and 1pm
df_sql6 = spark.sql("SELECT distinct r_version FROM SQL_table where time between '00:00:00' and '01:00:00'")
df_sql6.show()

+---------+
|r_version|
+---------+
|   2.15.1|
+---------+



In [58]:
#different OS are used
df_sql7 = spark.sql("SELECT distinct r_os FROM SQL_table ")
df_sql7.show()

+-----------+
|       r_os|
+-----------+
|         NA|
|    mingw32|
|  linux-gnu|
|darwin9.8.0|
+-----------+



In [81]:
#8.count different OS
df_sql8 = spark.sql("SELECT count(distinct r_os) as number_of_different_OS FROM SQL_table where r_os <> 'NA'")
df_sql8.show()


+----------------------+
|number_of_different_OS|
+----------------------+
|                     3|
+----------------------+

